In [1]:
# 操作 browser 的 API
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時的例外工具
from selenium.common.exceptions import TimeoutException

# 等待某個元素的出現
from selenium.webdriver.support.ui import WebDriverWait

# 期待元素出現並執行下一個指令
from selenium.webdriver.support import expected_conditions as EC

# 透過什麼方式選取元素
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

# 強制等待
from time import sleep

# 反爬蟲
import undetected_chromedriver as uc

# 執行 command 的時候用的
import os

# 整理 json 使用的工具
import json

# 計算程式執行的時間
import time

# 寫入csv檔
import csv

# 平行任務處理
from concurrent.futures import ThreadPoolExecutor as tpe

from fake_useragent import UserAgent 
ua = UserAgent(cache= True)

my_options = uc.ChromeOptions()

my_options.add_argument("--start-maximized") # 視窗最大化
my_options.add_argument("--incognito") # 無痕
my_options.add_argument("--disable-popup-blocking") # 禁用彈出攔截
my_options.add_argument("--disable-notifiactions") # 關閉推波通知
my_options.add_argument("--lang=zh-TW") # 設定為繁體中文
my_options.add_argument("--headless")
# my_options.add_argument("--user-agent="+ua.random)

# 建立儲存檔案的資料夾
folderPath = "BeitouRestaurant"
if os.path.exists(folderPath) != True:    
    os.makedirs(folderPath)
    
# 每搜尋一次計數器+1
index = 0

In [2]:
def page_link(link):
   
    global index
    
    # 自動取得Chrom driver 不用再額外取得 executable_path="chromedriver.exe"
    driver = uc.Chrome(options=my_options)
    
    # GoogleMap網址
    url = "https://www.google.com.tw/maps/@23.546162,120.6402133,8z?hl=zh-TW"
    
    driver.get(url)
    
    sleep(1)
    
    # 計算每跑一次跑多久
    start = time.time()
        
    # 輸入關鍵字搜尋
    driver.find_element(
        By.CSS_SELECTOR,
        "input#searchboxinput"
    ).send_keys(link)
    
    # 按下搜尋按鈕
    driver.find_element(
        By.CSS_SELECTOR,
        "button#searchbox-searchbutton"
    ).click()
        
    sleep (3)
        
    # 找搜尋元素結果的div
    focus = driver.find_elements(
        By.CSS_SELECTOR,
        "div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd"
    )[1]

    offset = 0
    innerHeight = 0
    count = 0  # 累計無效滾動次數
    limit = 5  # 最大無效滾動次數
    Flag = True
    counter = 0
        
    # 開始滾輪找到最底部
    while Flag:
            
        try:
            # 等待底部元素出現
            WebDriverWait(driver, 0.1).until(
                EC.presence_of_element_located( 
                    (By.CSS_SELECTOR, "span.HlvSq") 
                )
            )
            
            break
            
        except TimeoutException:

           # offset: 拉槓到頁面頂端的距離
            offset = driver.execute_script('return arguments[0].scrollTop', focus)

            # 執行js指令捲動頁面
            driver.execute_script('arguments[0].scrollTo(0, arguments[0].scrollHeight)', focus)
            
            # innerHeight: 頁面高度 = 拉槓到頁面頂端的距離
            innerHeight = driver.execute_script('return arguments[0].scrollHeight = arguments[0].scrollTop', focus)
            
            # 如果「拉槓到頁面頂端的距離」(offset)等於「頁面高度 = 拉槓到頁面頂端的距離」(innerHeight)，代表搜尋可能逾時
            if offset == innerHeight:
                count += 1                
              
            # 計數器等於限制數則搜尋確實逾時了,需重新整理並搜尋
            if count == limit:
                
                counter+=1
                
                print(f"{link}需重新整理,搜尋逾時,本次刷新第{counter}次")
                
                # 網頁重新整理
                driver.execute_script('window.location.replace(window.location.href)')
                
                # 重新搜尋
                driver.find_element(
                    By.CSS_SELECTOR,
                    "button#searchbox-searchbutton"
                 ).click()
                
                sleep(3)

                # 找搜尋結果的div
                focus = driver.find_elements(
                    By.CSS_SELECTOR,
                    "div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd"
                    )[1]

                # 重新定義需要的變數
                offset = 0
                innerHeight = 0
                count = 0  # 累計無效滾動次數
                
            sleep(1)
            
    # 尋找所有元素的href    
    href_ = driver.find_elements(
        By.CSS_SELECTOR,
        "div[data-js-log-root] a[aria-label]"
    )
      
    counter = 0 
    
    # 把所有href寫入csv裡
    for href in href_: 
        with open(f'{folderPath}/{folderPath}writer.csv', 'a', newline='') as f:
            writer = csv.writer(f)
            url = href.get_attribute("href")
            writer.writerow([url])   
        counter+=1
    
    # 清除搜尋的關鍵字
    driver.find_element(
        By.CSS_SELECTOR,
        "input#searchboxinput"
    ).clear()   
    
    # 每搜尋成功並寫入就計數器+1
    index+=1
    
    end = time.time()
    print(f'第{index}筆 {link},總共有:{counter}筆,已寫入完成')
    print("process_time 測量時間：%f 秒" % (end - start))
        
    driver.quit()
    
    # 每條(街or路)的全部restaurantType都抓完,則提示已完成全(街or路)抓取,,總共restaurantType有26種
    if index%26 == 0:
        regex01 = r'.+(街|路)'
        string01 = link
        match01 = re.search(regex01, string01)
        print(f'注意!!!,{match01[0]}:已全數寫入完成\n')
    else:
        print("繼續抓\n")
        


In [3]:
def multiprocess():
    
    district = "北投區" 
    
    roadNames = ["一德街","一心路","七星街","三合街１段","三合街２段","中和街","中和街新建巷","中和街錫安巷","中央北路１段","中央北路２段","中央北路３段","中央北路４段","中央南路１段","中央南路２段","中山北路７段","中山路","中心街","中正街","中興路","中華街","光明路","光明路新生巷","公舘路","勝利街","北投路１段","北投路２段","吉利街","同德街","大同街","大屯路","大屯路光華一巷","大屯路光華三巷","大屯路光華二巷","大屯路光華四巷","大度路","大度路怡和巷","大度路３段","大業路","大興街","天母北路","天母西路","奇岩路","奉賢路","學園路","宜山路","富貴一路","實踐街","尊賢街","崇仁路１段","崇仰一路","崇仰七路","崇仰三路","崇仰九路","崇仰二路","崇仰五路","崇仰六路","崗山路","幽雅路","幽雅路杏林巷","建國街","建民路","復興一路","復興三路","復興二路","復興四路","懷德街","承德路６段","承德路７段","振興街","振華街","文化三路","文林北路","新市街","新民路","新民路康樂巷","新民路香丘巷","新生街","新興路","明德路","杏林一路","杏林三路","杏林二路","東昇路","東華街１段","東華街２段","東陽街","榮華一路","榮華三路","榮華二路","樹林路","民族街","民權街","永興路１段","永興路２段","泉源路","泉源路華南巷","洲美街","清江路","湖山路１段","湖山路２段","湖底路","溫泉路","溫泉路天主巷","溫泉路湯元巷","溫泉路銀光巷","珠海路","登山路","知行路","石仙路","石牌路１段","石牌路２段","磺港路","福美路","秀山路","稻香路","立功街","立德路","立賢路","立農街１段","立農街２段","竹子湖路","紗帽路","義方街","義理街","翠宜路","翠嶺路","翠華街","翠雲街","聖景路","育仁路","自強街","致遠一路１段","致遠一路２段","致遠三路","致遠二路","行義路","裕民一路","裕民三路","裕民二路","裕民六路","裕民四路","西園街","西安街１段","西安街２段","豐年路１段","豐年路２段","進賢路","重三路","長壽路","開明街","關渡路","陽明路１段","陽明路２段","雙全街"]
    
    # 26種
    restaurantType = ["火鍋","拉麵","日式","美式","義式","法式","中式","台式","韓式","德式","地中海料理","印度料理","越式","港式","泰式","南洋","素食","鐵板燒","餐酒館","咖啡廳","熱炒店","早午餐","甜點店","燒肉","海鮮餐廳","牛排"]
    
    drt = []
    
    for road in roadNames:
        for type_ in restaurantType:
            keyword = f'{district}{road}{type_}' 
            # 平行運算x2,
            drt.append(keyword)
    
    with tpe(max_workers=2) as executor:
        executor.map(page_link,drt) # 執行page_link涵式,keyword是涵式需要的變數
        
        
#             with tpe(max_workers=3) as executor:
#                 executor.sumbit(page_link,keyword)
        

In [ ]:
if __name__=="__main__":
    multiprocess()

第1筆 北投區一德街拉麵,總共有:92筆,已寫入完成
process_time 測量時間：40.179486 秒
繼續抓

第2筆 北投區一德街火鍋,總共有:120筆,已寫入完成
process_time 測量時間：46.583566 秒
繼續抓

第3筆 北投區一德街日式,總共有:120筆,已寫入完成
process_time 測量時間：40.757730 秒
繼續抓

第4筆 北投區一德街美式,總共有:120筆,已寫入完成
process_time 測量時間：40.527236 秒
繼續抓

北投區一德街義式需重新整理,搜尋逾時,本次刷新第1次
北投區一德街中式需重新整理,搜尋逾時,本次刷新第1次
北投區一德街法式需重新整理,搜尋逾時,本次刷新第1次
北投區一德街德式需重新整理,搜尋逾時,本次刷新第1次
第5筆 北投區一德街韓式,總共有:112筆,已寫入完成
process_time 測量時間：46.645021 秒
繼續抓

北投區一德街地中海料理需重新整理,搜尋逾時,本次刷新第1次
第6筆 北投區一德街印度料理,總共有:120筆,已寫入完成
process_time 測量時間：40.191239 秒
繼續抓

第7筆 北投區一德街越式,總共有:120筆,已寫入完成
process_time 測量時間：40.274689 秒
繼續抓

第8筆 北投區一德街港式,總共有:120筆,已寫入完成
process_time 測量時間：42.140881 秒
繼續抓

北投區一德街泰式需重新整理,搜尋逾時,本次刷新第1次
北投區一德街南洋需重新整理,搜尋逾時,本次刷新第1次
北投區一德街泰式需重新整理,搜尋逾時,本次刷新第2次
北投區一德街南洋需重新整理,搜尋逾時,本次刷新第2次
第9筆 北投區一德街泰式,總共有:120筆,已寫入完成
process_time 測量時間：129.225605 秒
繼續抓

第10筆 北投區一德街南洋,總共有:120筆,已寫入完成
process_time 測量時間：101.935863 秒
繼續抓

北投區一德街素食需重新整理,搜尋逾時,本次刷新第1次
第11筆 北投區一德街餐酒館,總共有:122筆,已寫入完成
process_time 測量時間：42.051748 秒
繼續抓

第12筆 北投區一德街咖啡廳,總共有:105筆,已寫入完

第104筆 北投區中和街義式,總共有:120筆,已寫入完成
process_time 測量時間：40.656291 秒
第105筆 北投區中和街日式,總共有:105筆,已寫入完成
process_time 測量時間：94.565450 秒
繼續抓

繼續抓

第106筆 北投區中和街法式,總共有:46筆,已寫入完成
process_time 測量時間：8.505266 秒
繼續抓

第107筆 北投區中和街台式,總共有:32筆,已寫入完成
process_time 測量時間：12.851685 秒
繼續抓

第108筆 北投區中和街韓式,總共有:120筆,已寫入完成
process_time 測量時間：40.553144 秒
繼續抓

第109筆 北投區中和街德式,總共有:120筆,已寫入完成
process_time 測量時間：40.524478 秒
繼續抓

第110筆 北投區中和街地中海料理,總共有:115筆,已寫入完成
process_time 測量時間：38.825380 秒
繼續抓

第111筆 北投區中和街印度料理,總共有:120筆,已寫入完成
process_time 測量時間：40.596608 秒
繼續抓

第112筆 北投區中和街越式,總共有:120筆,已寫入完成
process_time 測量時間：40.604584 秒
第113筆 北投區中和街港式,總共有:75筆,已寫入完成
process_time 測量時間：27.850188 秒
繼續抓

繼續抓

第114筆 北投區中和街南洋,總共有:16筆,已寫入完成
process_time 測量時間：4.973706 秒
繼續抓

第115筆 北投區中和街泰式,總共有:106筆,已寫入完成
process_time 測量時間：37.273933 秒
繼續抓

第116筆 北投區中和街鐵板燒,總共有:8筆,已寫入完成
process_time 測量時間：3.314487 秒
繼續抓

第117筆 北投區中和街素食,總共有:120筆,已寫入完成
process_time 測量時間：40.614758 秒
繼續抓

北投區中和街咖啡廳需重新整理,搜尋逾時,本次刷新第1次
第118筆 北投區中和街餐酒館,總共有:120筆,已寫入完成
process_time 測量時間：40.826835 秒
繼續抓
